In [1]:
import pyspark
from delta import *

builder = pyspark.sql.SparkSession.builder.appName("MyApp") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

:: loading settings :: url = jar:file:/usr/local/lib/python3.10/dist-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/jupyter/.ivy2/cache
The jars for the packages stored in: /home/jupyter/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e38110d4-6be1-4bc4-8127-706ce9fcb9a7;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-spark_2.12/3.1.0/delta-spark_2.12-3.1.0.jar ...
	[SUCCESSFUL ] io.delta#delta-spark_2.12;3.1.0!delta-spark_2.12.jar (2903ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-storage/3.1.0/delta-storage-3.1.0.jar ...
	[SUCCESSFUL ] io.delta#delta-storage;3.1.0!delta-storage.jar (125ms)
downloading https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.9.3/antlr4-runtime-4.9.3.jar ...
	[SUCCESSFUL ] org.antlr#antlr4-runtime;4.9.3!antlr4-runtime.jar (159ms)
:: resolution report :: resolve 3681ms

In [2]:
df = spark.read.csv("alkon-hinnasto-tekstitiedostona_uusi.csv", header=True, inferSchema=True, sep=";")

df.select('Tyyppi').show()


+----------+
|    Tyyppi|
+----------+
|punaviinit|
|punaviinit|
|punaviinit|
|punaviinit|
|punaviinit|
|punaviinit|
|punaviinit|
|punaviinit|
|punaviinit|
|punaviinit|
|punaviinit|
|punaviinit|
|punaviinit|
|punaviinit|
|punaviinit|
|punaviinit|
|punaviinit|
|punaviinit|
|punaviinit|
|punaviinit|
+----------+
only showing top 20 rows



In [3]:
from pyspark.sql.functions import col

# Castataan tietotyypit
df = df.withColumn("Alkoholi-%", col("Alkoholi-%").cast("double"))
df = df.withColumn("Hapot g/l", col("Hapot g/l").cast("double"))
df = df.withColumn("Sokeri g/l", col("Sokeri g/l").cast("int"))
df = df.withColumn("Kantavierrep-%", col("Kantavierrep-%").cast("double"))
df = df.withColumn("Energia kcal/100 ml", col("Energia kcal/100 ml").cast("double"))

#Uudelleennimetään muutama huonosti nimetty sarake
df = df.withColumnRenamed("Hinnastojärjestyskoodi", "Hinnastojarjestyskoodi")
df = df.withColumnRenamed("Alkoholi-%", "Alkoholi_percent")
df = df.withColumnRenamed("Hapot g/l", "Hapot_g_l")
df = df.withColumnRenamed("Sokeri g/l", "Sokeri_g_l")
df = df.withColumnRenamed("Kantavierrep-%", "Kantavierrep_percent")
df = df.withColumnRenamed("Väri EBC", "Vari_EBC")
df = df.withColumnRenamed("Katkerot EBU", "Katkerot_EBU")
df = df.withColumnRenamed("Energia kcal/100 ml", "Energia_kcal_100_ml")

In [4]:
#Testataan, että skeema muuttui
df.printSchema()

root
 |-- Numero: string (nullable = true)
 |-- Nimi: string (nullable = true)
 |-- Valmistaja: string (nullable = true)
 |-- Pullokoko: string (nullable = true)
 |-- Hinta: string (nullable = true)
 |-- Litrahinta: string (nullable = true)
 |-- Uutuus: string (nullable = true)
 |-- Hinnastojarjestyskoodi: string (nullable = true)
 |-- Tyyppi: string (nullable = true)
 |-- Alatyyppi: string (nullable = true)
 |-- Erityisryhmä: string (nullable = true)
 |-- Oluttyyppi: string (nullable = true)
 |-- Valmistusmaa: string (nullable = true)
 |-- Alue: string (nullable = true)
 |-- Vuosikerta: string (nullable = true)
 |-- Etikettimerkintöjä: string (nullable = true)
 |-- Huomautus: string (nullable = true)
 |-- Rypäleet: string (nullable = true)
 |-- Luonnehdinta: string (nullable = true)
 |-- Pakkaustyyppi: string (nullable = true)
 |-- Suljentatyyppi: string (nullable = true)
 |-- Alkoholi_percent: double (nullable = true)
 |-- Hapot_g_l: double (nullable = true)
 |-- Sokeri_g_l: integer 

In [5]:
 #Filteröidään punaiset ja valkoiset viinit
punaviinit_df = df.filter(df["Tyyppi"] == "punaviinit")
valkoviinit_df = df.filter(df["Tyyppi"] == "valkoviinit")

# Kirjoitetaan punaviinit ja valkoviinit omiin delta-tauluihin. Merge schema siksi, että mikäli tulee uusia sarakkeita tms, niin delta hoitaa sen.
punaviinit_df.write.format("delta").mode("overwrite").option("mergeSchema", "true").save("data/punaviinit_table")
valkoviinit_df.write.format("delta").mode("overwrite").option("mergeSchema", "true").save("data/valkoviinit_table")


24/03/07 10:07:24 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [6]:
#Katsotaan, että taulusta löytyy tavaraa

df = spark.read.format("delta").load("data/punaviinit_table")
df.count()



3115

In [7]:
#Katsotaan, että taulusta löytyy tavaraa
df = spark.read.format("delta").load("data/valkoviinit_table")
df.count()

2202